In [3]:
#  источники - https://www.kaggle.com/code/pierremegret/gensim-word2vec-tutorial/notebook
# https://spacy.io/models/ru#ru_core_news_md

import re  # Для препроцессинга
!pip install pandas
import pandas as pd  # Для обработки данных
from time import time  # Для определения времени операций
import spacy
from spacy.lang.ru.examples import sentences 
from collections import defaultdict  # Для частоты слов
import logging  # Настройка для gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [5]:
# Считываю файл
data = pd.read_csv('Сlean.csv')
df=data['description']

In [6]:
df.head(-5)

0         Прекрасная однокомнатная квартира в доме с лиф...
1         Светлая, просторная комната 19 кв. метров. Отд...
2         Первомайская, 15 мин. пешкомТёплая уютная квар...
3         в шаговой доступности парк Сокольники. , Разви...
4         Ильинское с.Все коммуникации центральные.Кварт...
                                ...                        
100713    Переуступка, входит в цену, 2 корпус, комнаты ...
100714    Продается трехкомнатная квартира, замечательно...
100715    ПРОЧИТАЙТЕ ОБЪЯВЛЕНИЕ ПОЛНОСТЬЮ !!!Звонить СТР...
100716    Сначала читаем, потом звоним.)Согласно статье ...
100717    ПРОЧИТАЙТЕ ОБЪЯВЛЕНИЕ ПОЛНОСТЬЮ !!!Звонить СТР...
Name: description, Length: 100718, dtype: object

In [7]:
df.isnull().sum()

0

In [8]:
# Загружаю модель
!python -m spacy download ru_core_news_sm

ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'C:\\Users\\Daniel\\AppData\\Local\\Temp\\pip-uninstall-yljdjjbs\\annotated_types.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




     ---------------------------------------- 15.3/15.3 MB 7.7 MB/s eta 0:00:00
  Using cached pydantic-1.8.2-cp39-cp39-win_amd64.whl (1.9 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.2
    Uninstalling pydantic-1.10.2:
      Successfully uninstalled pydantic-1.10.2


In [9]:
nlp = spacy.load('ru_core_news_sm', disable=['ner', 'parser']) # отключение распознавания именованных объектов для повышения скорости
def cleaning(doc):
    #Лемматизируется и удаляет стоп-слова
    # doc должен быть пространственным объектом Doc
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec использует контекстные слова для изучения векторного представления целевого слова,
    # если предложение состоит всего из одного или двух слов,
    # польза от тренинга мала, поэтому
    if len(txt) > 2:
        return ' '.join(txt)

INFO - 21:50:48: Loading dictionaries from C:\Users\Daniel\anaconda3\lib\site-packages\pymorphy2_dicts_ru\data
INFO - 21:50:48: format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


In [10]:
    #Удаляю неалфавитные символы:
brief_cleaning = (re.sub("[^А-Яа-я]", ' ', str(row)).lower() for row in data['description'])

In [11]:
# Атрибут space .pipe() для ускорения процесса очистки:
# Подбираю оптимальное batch_size=2000, n_process=4
t = time()
txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=2000, n_process=4)]
print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 25.37 mins


In [ ]:
#Помещаю результаты в фрейм данных, чтобы удалить недостающие значения и дубликаты:
df_clean = pd.DataFrame({'clean': txt})
#df_clean = df_clean.dropna().drop_duplicates()

In [12]:
df_clean = pd.DataFrame({'clean': txt})

In [13]:
df_clean.head(-5)

,clean
0,прекрасный однокомнатный квартира дом лифт мус...
1,светлый просторный комната кв метр о...
2,первомайский мина пешкомт плая уютный к...
3,шаговый доступность парк сокольники развит...
4,ильинский коммуникация центральный квартира хо...
...,...
100713,переуступка входить цена корпус комнат...
100714,продаваться трехкомнатная квартира замечател...
100715,прочитайте объявление полностью звонить ст...
100716,читать звоним согласно статья фз и...


In [14]:
#Удаляю пробелы которых больше или равно 2 и сохраняю данные
df_clean['clean']= df_clean['clean'].str.replace(' {2,}', ' ', regex=True)
df_clean.to_csv('PRE.csv',index=False)

In [15]:
df_clean.head(-5)

,clean
0,прекрасный однокомнатный квартира дом лифт мус...
1,светлый просторный комната кв метр отдельный л...
2,первомайский мина пешкомт плая уютный квартира...
3,шаговый доступность парк сокольники развитой и...
4,ильинский коммуникация центральный квартира хо...
...,...
100713,переуступка входить цена корпус комната изолир...
100714,продаваться трехкомнатная квартира замечательн...
100715,прочитайте объявление полностью звонить строго...
100716,читать звоним согласно статья фз исполнительны...
